In [1]:
!pip install tavily-python
!pip install langchain_openai
from langchain_openai import ChatOpenAI
!pip install langchain_community
from langchain_community.tools.tavily_search import TavilySearchResults

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [2]:
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

In [3]:
pip install gradio langchain openai langchain-community python-dotenv

In [4]:
import os
os.environ['OPENAI_API_KEY']='secert'
os.environ['TAVILY_API_KEY']='secret'

In [6]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import gradio as gr
import os
from dotenv import load_dotenv

# Load keys from .env
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Pydantic model
class InfoSearch(BaseModel):
    first_name: Optional[str] = Field(default=None)
    last_name: Optional[str] = Field(default=None)
    designation: Optional[str] = Field(default=None)
    company_name: Optional[str] = Field(default=None)
    email_id: Optional[str] = Field(default=None)
    user_query: Optional[str] = Field(default=None)
    location: Optional[str] = Field(default=None)

# Core logic
def user_and_company_info_research(details: InfoSearch) -> str:
    try:
        tavily_search = TavilySearchResults(
            max_results=1,
            tavily_api_key=TAVILY_API_KEY,
            include_domains=["linkedin.com"],
            include_raw_content=True
        )

        search_prompt = ""
        for key, value in details.model_dump().items():
            if value and key != "user_query":
                search_prompt += f"{value}, "

        results = tavily_search.invoke({"query": search_prompt})
        print("🔍 Tavily result:", results)

        if not results or "content" not in results[0]:
            return "No relevant results found or content unavailable."

        model = ChatOpenAI(
            model="gpt-4o-mini",
            openai_api_key=OPENAI_API_KEY
        )

        query_cons = HumanMessage(content=f"""
            Reference content: {results[0]["content"]}
            User query: {details.user_query}
        """)

        messages = [
            SystemMessage(content=(
                "You answer information about a professional. Be polite, formal, and fact-based using LinkedIn content."
            )),
            query_cons
        ]

        result = model.invoke(messages)
        return result.content

    except Exception as e:
        print("ERROR:", e)
        return f" An error occurred: {str(e)}"

# Gradio wrapper
def gradio_interface(first_name, last_name, designation, company_name, email_id, user_query, location):
    try:
        info = InfoSearch(
            first_name=first_name,
            last_name=last_name,
            designation=designation,
            company_name=company_name,
            email_id=email_id,
            user_query=user_query,
            location=location
        )
        return user_and_company_info_research(info)
    except Exception as e:
        print(" Input error:", e)
        return f" Invalid input: {str(e)}"

In [18]:
with gr.Blocks(css="""
    .gradio-container {
        background: linear-gradient(to bottom right, #fdf6f0, #e0f2fe);
        font-family: 'Segoe UI', sans-serif;
        padding-bottom: 50px;
    }
    .gr-button {
        background: linear-gradient(to right, #38bdf8, #0ea5e9) !important;
        color: white !important;
        border-radius: 10px;
        font-weight: 600;
        font-size: 16px;
        padding: 12px 24px;
        box-shadow: 0 4px 14px rgba(0,0,0,0.08);
    }
    .gr-textbox label {
        font-weight: 600;
        font-size: 14px;
        color: #334155;
    }
    .gr-textbox textarea, .gr-textbox input {
        border-radius: 8px !important;
        border: 1px solid #cbd5e1;
        box-shadow: none !important;
        background-color: #1e293b;
    }
    #header {
        text-align: center;
        padding: 40px 0 10px 0;
    }
    .card-output {
        border: 1px solid #1e293b;
        font-weight: 600;
        border-radius: 12px;
        padding: 20px;
        background: white;
        box-shadow: 0 4px 18px rgba(0,0,0,0.05);
        font-size: 20px;
        color: #1e293b;
        line-height: 1.6;
    }
    .card-output .svelte-1ipelgc label {
        color: #1e293b !important;
        opacity: 1 !important;
        font-weight: 600;
    }

""") as demo:
    gr.HTML("""
        <div id="header">
            <h1 style="font-size: 32px; color:#0ea5e9; margin-bottom: 8px;">Professional Lookup Assistant</h1>
            <p style="font-size: 16px; color:#64748b;">AI-powered insights from professional profiles across the web</p>
        </div>
    """)

    with gr.Row():
        with gr.Column():
            first_name = gr.Textbox(label="👤 First Name")
            last_name = gr.Textbox(label="👤 Last Name")
            designation = gr.Textbox(label="💼 Designation")
            location = gr.Textbox(label="📍 Location")
        with gr.Column():
            company_name = gr.Textbox(label="🏢 Company Name")
            email_id = gr.Textbox(label="📧 Email ID")
            user_query = gr.Textbox(label="❓ Your Question", lines=3, placeholder="e.g. What is the company '--' do?")

    search_button = gr.Button("🔎 Search Now")
    output_box = gr.Textbox(label="AI-Powered Insight", lines=10, interactive=False, elem_classes=["card-output"])

    search_button.click(fn=gradio_interface,
                        inputs=[first_name, last_name, designation, company_name, email_id, user_query, location],
                        outputs=output_box)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://940430db407454d3c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
